In [3]:
import gzip
import scipy
import scipy.optimize
import random
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import array
import gzip
import random
from collections import defaultdict

In [4]:
def readJSON(path):
    for l in gzip.open(path, 'rt', encoding='UTF-8'):
        d = eval(l)
        u = d['userID']
        try:
            g = d['gameID']
        except Exception as e:
            g = None
        yield u, g, d

## Question 1 (Baseline)

In [443]:
#Splitting into training and validation sets
raw_data = []
for user, game, d in readJSON("train.json.gz"):
    raw_data.append([user, game, d])

random.shuffle(raw_data)    
    
raw_train = raw_data[:165000]
raw_valid = raw_data[165000:]

In [444]:
# sentiDict = {}
# for line in raw_data:
#     t = line[2]['text']
#     senti1 = vader.polarity_scores(t)
#     senti = []
#     senti.append(senti1['neg'])
#     senti.append(senti1['neu'])
#     senti.append(senti1['pos'])
#     senti.append(senti1['compound'])
    
#     tmp = line[2]['early_access']
#     e = 0
#     if tmp == True:
#         e = 1
    
#     senti.append(e)
    
#     sentiDict[line[1]] = senti

In [445]:
#Creating a dict with user as key and all games played by him as values
gamesOfUser = defaultdict(set)
for item in raw_data:
    gamesOfUser[item[0]].add(item[1])

#Creating a set with all the games
allGames = set()
# allGamesList = sorted(list(allGames))
# allGamesDict = {}
# for i, g in enumerate(allGamesList):
#     allGamesDict[g] = i/len(allGamesList)
    
for item in raw_data:
    allGames.add(item[1])

In [446]:
trainPos = []
for item in raw_train:
    tup = (item[0], item[1], 1)
    trainPos.append(tup)
    
trainNeg = []
for z in range(1):
    for item in raw_train:
        user = item[0]
        games = gamesOfUser[user]
        notPlayedGames = allGames - games
        randomOne = random.sample(notPlayedGames, 1)[0]
        tup = (user, randomOne, 0)
        trainNeg.append(tup)

len(trainPos), len(trainNeg)

(165000, 165000)

In [429]:
#Modifying validation set
valid = []
for item in raw_valid:
    tup = (item[0], item[1], 1)
    valid.append(tup)

#Creating a negative validation elements and appending to validation set (valid)
for item in raw_valid:
    user = item[0]
    games = gamesOfUser[user]
    notPlayedGames = allGames - games
    randomOne = random.sample(notPlayedGames, 1)[0]
    tup = (user, randomOne, 0)
    valid.append(tup)    

In [430]:
### Would-play baseline: just rank which games are popular and which are not, and return '1' if a game is among the top-ranked

gameCount = defaultdict(int)
totalPlayed = 0

for user, game, _ in raw_train:
    gameCount[game] += 1
    totalPlayed += 1

mostPopular = [(gameCount[x], x) for x in gameCount]
mostPopular.sort(reverse = True)

maxi = 0
for item in mostPopular:
    maxi = max(maxi, item[0])

mostPopularDict = defaultdict(float)
for item in mostPopular:
    mostPopularDict[item[1]] = item[0]/maxi

In [431]:
return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPlayed/2: break

predictions = []

for l in valid:
    u, g, _ = l[0], l[1], l[2]
    if g in return1:
        predictions.append(1)
    else:
        predictions.append(0)

actualValues = []
for item in valid:
    actualValues.append(item[2])
    

correctValues = [a == b for a, b in zip(predictions, actualValues)]

print('Validation Accuracy: ' + str(sum(correctValues)/len(valid)*100))

Validation Accuracy: 67.905


## Question 2 (Popularity)

In [432]:
return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    #Taking 68.9% of the top values (1/1.45)
    if count > totalPlayed/1.45: break

predictions = []

for l in valid:
    u, g, _ = l[0], l[1], l[2]
    if g in return1:
        predictions.append(1)
    else:
        predictions.append(0)

actualValues = []
for item in valid:
    actualValues.append(item[2])
    

correctValues = [a == b for a, b in zip(predictions, actualValues)]

print('Validation Accuracy: ' + str(sum(correctValues)/len(valid)*100))

Validation Accuracy: 69.925


## Question 3 (Jaccard)

In [433]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

In [434]:
#Creating 2 dicts. One with game and one with user

gamesOfUser = defaultdict(set)
usersOfGame = defaultdict(set)

#Only on training set
for item in raw_train:
    gamesOfUser[item[0]].add(item[1])
    usersOfGame[item[1]].add(item[0])

In [440]:
def mostSimilar(u, g):
    similarities1 = []
    users = usersOfGame[g]
    for g2 in gamesOfUser[u]:
        if g == g2: continue # other than the query
        sim = Jaccard(users, usersOfGame[g2])
        similarities1.append(sim)
    
#     count = 0
#     threshold = 0.008
#     for sim in similarities1:
#         if sim > threshold:
#             count += 1
    
#     if count > 0.5 * len(similarities1):
#         return 1

    if len(similarities) == 0:
        return 
    
    return 0

In [ ]:
def mostSimilar(u, g):
    similarities1 = []
    users = usersOfGame[g]
    for g2 in gamesOfUser[u]:
        if g == g2: continue # other than the query
        sim = Jaccard(users, usersOfGame[g2])
        similarities1.append(sim)
    
#     similarities2 = []
#     games = gamesOfUser[u]
#     for u2 in usersOfGame[g]:
#         if u == u2: continue # other than the query
#         sim = Jaccard(games, gamesOfUser[u2])
#         similarities2.append(sim)
    
#     count = 0
#     #Thershold set as 0.008
#     threshold = 0.008
#     for sim in similarities:
#         if sim > threshold:
#             count += 1
    
    #Since it is given most of the values should cross. Assuming it to be more than 50%.
    #Can be done using median too
#     if count > 0.5*len(similarities):
#         return 1

    if len(similarities1) > 0:
        min1 = min(similarities1)
    else:
        min1 = 0.002459177358905974
    
    
    
    max1 = max(similarities1) if len(similarities1) > 0 else 0.036477232467427144
    mean1 = sum(similarities1)/len(similarities1) if len(similarities1) > 0 else 0.013651729456640182
    #median1 = median(similarities1) if len(similarities1) > 0 else 0
    
#     min2 = min(similarities2) if len(similarities2) > 0 else 0.008970185399297994
#     max2 = max(similarities2) if len(similarities2) > 0 else 0.0960382978251585
#     mean2 = sum(similarities2)/len(similarities2) if len(similarities2) > 0 else 0.03420200386502156
    #median2 = median(similarities2) if len(similarities2) > 0 else 0
    
    pop = mostPopularDict[g]
    if pop == 0:
        pop = 0.12058540079119132
    
#     senti = [0, 0, 0, 0]
#     if g in sentiDict:
#         senti = sentiDict[g]
    
    return [1, pop, max1, min1, mean1] #, min2, max2, mean2+ senti

In [ ]:
trainX = []
trainY = []
for tup in trainPos:
    trainX.append(mostSimilar(tup[0], tup[1]))
    trainY.append(tup[2])
    
for tup in trainNeg:
    trainX.append(mostSimilar(tup[0], tup[1]))
    trainY.append(tup[2])

In [ ]:
# min1All = 0
# max1All = 0
# mean1All = 0
# min2All = 0
# max2All = 0
# mean2All = 0
# popAll = 0
# countpop = 0
# count1 = 0
# count2 = 0

# for item in trainX:
#     if item[1] != 0:
#         countpop += 1
#         popAll += item[1]
#     if item[2] != 0:
#         count1 += 1
#         max1All += item[2]
#         min1All += item[3]
#         mean1All += item[4]
#     if item[5] != 0:
#         count2 += 1
#         max2All += item[6]
#         min2All += item[5]
#         mean2All += item[7]

In [ ]:
# min1All /= count1
# max1All /= count1
# mean1All /= count1

# min2All /= count2
# max2All /= count2
# mean2All /= count2

# popAll /= countpop

In [ ]:
# print(min1All, max1All, mean1All, min2All, max2All, mean2All, popAll)

In [ ]:
# for item in trainX:
#     if item[1] == 0:
#         item[1] = popAll
#     if item[2] == 0:
#         item[2] = max1All
#         item[3] = min1All
#         item[4] = mean1All
#     if item[5] == 0:
#         item[6] = max2All
#         item[5] = min2All
#         item[7] = mean2All

In [ ]:
ntrainX = np.matrix(trainX)
ntrainy = np.array(trainY).T
ntrainX.shape, ntrainy.shape

In [ ]:
validX = []
for item in valid:
    validX.append(mostSimilar(item[0], item[1]))

In [ ]:
import sklearn
from sklearn.linear_model import LogisticRegression

reg = sklearn.linear_model.LogisticRegression(C = 0.05, class_weight='balanced')
reg.fit(ntrainX, ntrainy)

In [ ]:
validX = np.matrix(validX)
predictions = reg.predict(validX)

In [ ]:
actualValues = []
for item in valid:
    actualValues.append(item[2])    

correctValues = [a == b for a, b in zip(predictions, actualValues)]

print('Validation Accuracy: ' + str(sum(correctValues)/len(valid)*100))

In [ ]:
f = open('pairs_Played.txt', 'r')
lines = f.readlines()
f.close()

f = open('pairs_Played_Out.txt', 'w')
f.write('userID-gameID,prediction\n')
lines = lines[1:]
for line in lines:
    u, g = line.strip().split('-')
    testX = mostSimilar(u, g)
    testX = np.matrix(testX)
    res = reg.predict(testX)
    f.write(u + '-' + g + ',' + str(res)[1:2] + '\n')

f.close()

## Question 4 (Both Jaccard and Popularity)

In [441]:
out = 0
from statistics import median
predictions = []
for item in valid:
    #Only checking Jaccard if it is popular in the first place
    if item[1] in return1:
        predictions.append(mostSimilar(item[0], item[1]))
    else:
        predictions.append(0)
    
actualValues = []
for item in valid:
    actualValues.append(item[2])    

correctValues = [a == b for a, b in zip(predictions, actualValues)]

print('Validation Accuracy: ' + str(sum(correctValues)/len(valid)*100))

Validation Accuracy: 71.93


## Question 5 (Test Set and Submitting to Kaggle)

In [ ]:
f = open('pairs_Played.txt', 'r')
lines = f.readlines()
f.close()

f = open('pairs_Played_Final.txt', 'w')
f.write('userID-gameID,prediction\n')
lines = lines[1:]
for line in lines:
    u, g = line.strip().split('-')
    if g in return1:
        res = mostSimilar(u, g)
    else:
        res = 0
    f.write(u + '-' + g + ',' + str(res) + '\n')

f.close()

In [ ]:
print('Submitted to Kaggle')
print('\nKaggle Username: aditya1c')

## Question 9

In [14]:
import gzip
import scipy
import scipy.optimize
import random
import numpy as np
from collections import defaultdict
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [15]:
import tensorflow as tf
import array
import gzip
import random
from tensorflow.keras import Model
from collections import defaultdict

In [5]:
def readJSON1(path):
    for l in gzip.open(path, 'rt', encoding='UTF-8'):
        d = eval(l)
        u = d['userID']
        try:
            g = d['gameID']
        except Exception as e:
            g = None
        yield u, g, d

In [28]:
#Splitting into training and validation sets
dataset = []
for user, game, d in readJSON1("train.json.gz"):
    dataset.append([user, game, d['hours_transformed']])

# random.shuffle(dataset)    
    
train = dataset[:175000]
valid = dataset[165000:]

In [29]:
userIDs = {}
itemIDs = {}
interactions = []
intValid = []

# Could adapt to any dataset, this one is from
for ele in train:
    u = ele[0]
    i = ele[1]
    r = ele[2]
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
    interactions.append((u,i,r))
    
for ele in valid:
    u = ele[0]
    i = ele[1]
    r = ele[2]
    intValid.append((u,i,r))

In [412]:
# mean rating, just for initialization
mu = sum([r for _,_,r in interactions]) / len(interactions)

In [413]:
# Gradient descent optimizer, experiment with learning rate
optimizer = tf.keras.optimizers.Adam(0.1)

In [414]:
class LatentFactorModel(tf.keras.Model):
    def __init__(self, mu, K, lamb1, lamb2):
        super(LatentFactorModel, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.1))
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.1))
        
        self.betaU.assign(list(userBiases.values()))
        self.betaI.assign(list(gameBiases.values()))
        
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.1))
        self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.1))
        self.lamb1 = lamb1
        self.lamb2 = lamb2

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i] +\
            tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb1 * (tf.reduce_sum(self.betaU**2) + tf.reduce_sum(self.betaI**2)) + self.lamb2 * (tf.reduce_sum(self.gammaU**2) + tf.reduce_sum(self.gammaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        pred = self.alpha + beta_u + beta_i +\
               tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)

In [420]:
# Experiment with number of factors and regularization rate
model = LatentFactorModel(3.1164, 8, 0.00001, 0.005)

In [421]:
def trainingStep(interactions):
    Nsamples = 175000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactions)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleR.append(r)

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

In [422]:
# 100 iterations of gradient descent
for i in range(80):
    obj = trainingStep(interactions)
    print("iteration " + str(i) + ", objective = " + str(obj))

iteration 0, objective = 5.109024
iteration 1, objective = 2.8393712
iteration 2, objective = 2.6481733
iteration 3, objective = 3.6970587
iteration 4, objective = 3.3134131
iteration 5, objective = 2.5834265
iteration 6, objective = 2.5802207
iteration 7, objective = 2.540879
iteration 8, objective = 2.2537599
iteration 9, objective = 2.2860477
iteration 10, objective = 2.3310444
iteration 11, objective = 2.1392677
iteration 12, objective = 1.9966738
iteration 13, objective = 2.0006428
iteration 14, objective = 1.9733336
iteration 15, objective = 1.8927789
iteration 16, objective = 1.837797
iteration 17, objective = 1.8162799
iteration 18, objective = 1.7826585
iteration 19, objective = 1.7538829
iteration 20, objective = 1.712073
iteration 21, objective = 1.6775392
iteration 22, objective = 1.681324
iteration 23, objective = 1.645169
iteration 24, objective = 1.6233385
iteration 25, objective = 1.604901
iteration 26, objective = 1.6020132
iteration 27, objective = 1.5819721
iteration

In [418]:
pred = []
actual = []
for u, g, h in intValid:
    pred.append(model.predict(userIDs[u], itemIDs[g]).numpy())
    actual.append(h)
    
mse = []
for i in range(len(pred)):
    mse.append((pred[i] - actual[i])**2)
    
mse = sum(mse)/len(mse)

mse

2.8073114928465777

In [423]:
f = open('pairs_Hours.txt', 'r')
lines = f.readlines()
f.close()

f = open('pairs_Hours_Out.txt', 'w')
f.write('userID-gameID,prediction\n')
lines = lines[1:]
for line in lines:
    u, g = line.strip().split('-')
    res = model.predict(userIDs[u], itemIDs[g]).numpy()
    #This can be replaced with 0 if required too. Just used for checking in Kaggle
#     if u not in userBiases or g not in gameBiases:
#         res = ratingMean
        
    f.write(u + '-' + g + ',' + str(res) + '\n')

f.close()

In [30]:
hoursPerUser = defaultdict(list) 
hoursPerGame = defaultdict(list)
allHours = []

for user, game, h in train:
    allHours.append(h)
    #These variables are not used much. Just to maintain consistency with workbook code
    hoursPerUser[user].append(h)
    hoursPerGame[game].append(h)

In [31]:
N = len(dataset)
nUsers = len(hoursPerUser)
nGames = len(hoursPerGame)
users = list(hoursPerUser.keys())
games = list(hoursPerGame.keys())

In [32]:
alpha = sum(allHours)/len(allHours)
userBiases = defaultdict(float)
gameBiases = defaultdict(float)

Defining Functions which are similar to those discussed in the class

In [33]:
def prediction(user, game):
    return alpha + userBiases[user] + gameBiases[game]

In [34]:
def unpack(theta):
    global alpha
    global userBiases
    global gameBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    gameBiases = dict(zip(games, theta[1+nUsers:]))

In [35]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(u, g) for u, g, h in dataset]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in gameBiases:
        cost += lamb*gameBiases[i]**2
    return cost

In [36]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(dataset)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dGameBiases = defaultdict(float)
    for u, g, h in dataset:
        u,i = u, g
        pred = prediction(u, i)
        diff = pred - h
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dGameBiases[i] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in gameBiases:
        dGameBiases[i] += 2*lamb*gameBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dGameBiases[i] for i in games]
    return np.array(dtheta)

In [37]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

Training on training set with lambda = 1

In [38]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nGames),
                             derivative, args = (allHours, 0.000029))

MSE = 5.280231958987971
MSE = 5.161276766723984
MSE = 4.75220537977447
MSE = 15.497401226086064
MSE = 4.559106199125477
MSE = 4.07024982602844
MSE = 4.057352528717888
MSE = 4.00751662481554
MSE = 3.8362255746403258
MSE = 3.495404784862
MSE = 3.3151372824830214
MSE = 3.110933249679368
MSE = 3.026955091253423
MSE = 2.931702174551582
MSE = 2.875921278902824
MSE = 2.838281637606661
MSE = 2.8270300833697988
MSE = 2.8165051834919237
MSE = 4.138396012442667
MSE = 2.816345248025124
MSE = 2.809047274049851
MSE = 2.7975875941133492
MSE = 2.796215105085698
MSE = 2.7938008029820147
MSE = 2.7909121016047473
MSE = 2.788390937912183
MSE = 2.7857278840085944
MSE = 2.7864035436939614
MSE = 2.785643915804261
MSE = 2.7854691499232236
MSE = 2.7852270314773064
MSE = 2.786018988600938
MSE = 2.785224958222594
MSE = 2.784961537714502
MSE = 2.78491255777868
MSE = 2.7848542453288756
MSE = 2.7848438273236487
MSE = 2.7844494040317658
MSE = 2.784492887688483
MSE = 2.784405497814387
MSE = 2.784205426490971
MSE = 2.

(array([ 3.11527399, -1.1599159 ,  0.41032202, ..., -0.27174604,
        -0.73349262, -1.62234258]),
 2.9206101252145085,
 {'grad': array([-6.07451834e-06, -3.88092616e-07, -5.96837378e-08, ...,
         -4.94227630e-08,  2.38215942e-08, -4.98864542e-07]),
  'task': b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'funcalls': 124,
  'nit': 100,
  'warnflag': 0})

In [39]:
predictionsValid = []
for u, g, h in valid:
    predictionsValid.append(prediction(u, g))

In [40]:
mseValid = MSE(predictionsValid, allHoursValid)
print('MSE on Validation Set with lambda as 1 is ' + str(mseValid))

MSE on Validation Set with lambda as 1 is 2.749940733410276


In [41]:
f = open('pairs_Hours.txt', 'r')
lines = f.readlines()
f.close()

f = open('pairs_Hours_Out_2.txt', 'w')
f.write('userID-gameID,prediction\n')
lines = lines[1:]
for line in lines:
    u, g = line.strip().split('-')
    res = prediction(u, g)
    #This can be replaced with 0 if required too. Just used for checking in Kaggle
    if u not in userBiases or g not in gameBiases:
        res = ratingMean
        
    f.write(u + '-' + g + ',' + str(res) + '\n')

f.close()

## Question 10

In [31]:
#Sorting using lambda and key
sorted_users = sorted(userBiases.items(), key = lambda x: x[1])
smallestUser = sorted_users[0]
largestUser = sorted_users[-1]

In [32]:
sorted_games = sorted(gameBiases.items(), key = lambda x: x[1])
smallestGame = sorted_games[0]
largestGame = sorted_games[-1]

In [33]:
print('Smallest User Bias: ' + str(smallestUser))
print('\nLargest User Bias: ' + str(largestUser))

Smallest User Bias: ('u04589037', -3.03607644319556)

Largest User Bias: ('u38845867', 6.2844331857368285)


In [34]:
print('Smallest Game Bias: ' + str(smallestGame))
print('\nLargest Game Bias: ' + str(largestGame))

Smallest Game Bias: ('b05546112', -2.8090516714749536)

Largest Game Bias: ('b80191205', 5.534327809661798)


## Question 11

Testing for lambda from 0 to 1 in steps of 0.05

In [35]:
bestLambda = 0
bestMSE = float('inf')
lambdaList = []
mseList = []

lamb = 0
step = 0.05
#Training and checking using Lambda from 0 to 1
for i in range(int(1/step) + 1):
    tmpLamb = lamb + 0.05*i
    scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nGames),
                             derivative, args = (allHours, tmpLamb), disp = False)
    predictionsValid = []
    for u, g, h in valid:
        predictionsValid.append(prediction(u, g))
    mseValid = MSE(predictionsValid, allHoursValid)
    lambdaList.append(tmpLamb)
    mseList.append(mseValid)
    if mseValid < bestMSE:
        bestMSE = mseValid
        bestLambda = tmpLamb

MSE = 5.696797120965593
MSE = 5.3868176525322236
MSE = 5.262014825224892
MSE = 5.254219187298708
MSE = 5.225676100566427
MSE = 4.946000296618459
MSE = 4.575342529889029
MSE = 4.204257517553415
MSE = 4.0251737458044134
MSE = 3.9696241872528866
MSE = 3.9515104511061407
MSE = 3.8985446906103958
MSE = 3.807113313669612
MSE = 3.6636070522601547
MSE = 3.5417041934016296
MSE = 3.339520688184969
MSE = 3.2893828833599033
MSE = 3.2006075872201283
MSE = 3.1447864417237033
MSE = 3.113303618570252
MSE = 3.0936696227260625
MSE = 3.7636476670203343
MSE = 3.085245997819632
MSE = 3.0561949440034257
MSE = 2.984427315945522
MSE = 2.946546266980766
MSE = 2.9009488975906375
MSE = 3.0971629162698884
MSE = 2.8936863021866426
MSE = 2.8769489790075893
MSE = 2.854174723086478
MSE = 2.8452075420192053
MSE = 2.8407238983980836
MSE = 2.838215237375079


KeyboardInterrupt: 

The above code block output is run locally but not showed it here as the output is too long

In [ ]:
print('Best Lambda: ' + str(bestLambda))
print('Best MSE: ' + str(bestMSE))

In [ ]:
plt.plot(lambdaList, mseList)
plt.xlabel('Lambda')
plt.ylabel('MSE')
plt.title('Variation of MSE with Lambda')

Best Model is used to predict the test set and uploaded to Kaggle

In [373]:
f = open('pairs_Hours.txt', 'r')
lines = f.readlines()
f.close()

f = open('pairs_Hours_Out_2.txt', 'w')
f.write('userID-gameID,prediction\n')
lines = lines[1:]
for line in lines:
    u, g = line.strip().split('-')
    res = prediction(u, g)
    #This can be replaced with 0 if required too. Just used for checking in Kaggle
    if u not in userBiases or g not in gameBiases:
        res = ratingMean
        
    f.write(u + '-' + g + ',' + str(res) + '\n')

f.close()

In [ ]:
print('Submitted to Kaggle')
print('\nKaggle Username: aditya1c')